In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
from supervised.automl import AutoML
import category_encoders as ce
import math
import re

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
print('Imported sci-kit learn libraries!')

Imported sci-kit learn libraries!


## Now I am going to try clean my data down further regarding the neighbours column
I will be making some changes manually as I see fit but will try take a more sophisticated approach when I can.

In [2]:
irl = pd.read_csv('neighbours.csv', index_col='Unnamed: 0')

In [3]:
irl

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
0,"5 Braemor Drive, Churchtown",NaN,Dublin,343000.0,1,1,1,0,2010,1,Churchtown Dublin,Dublin,0,Braemor Drive
1,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,185000.0,1,0,0,1,2010,1,Portlaoise,Midlands,0,Summerhill Lane
2,"1 Meadow Avenue, Dundrum",Dublin 14,Dublin,438500.0,1,1,1,1,2010,1,Dundrum Dublin,Dublin,0,Meadow Avenue
3,"1 The Haven, Mornington",NaN,Meath,400000.0,1,1,1,1,2010,1,Mornington,Mid-East,0,The Haven
4,"12 Sallymount Avenue, Ranelagh",NaN,Dublin,425000.0,1,1,1,1,2010,1,Ranelagh,Dublin,0,Sallymount Avenue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308454,"24 North Street, Skibereen, Co Cork",NaN,Cork,30000.0,1,1,1,1,2016,4,Skibbereen,South-West (IE),0,North Street
308455,"Tooreen, Skibereen, Co Cork",NaN,Cork,70000.0,1,1,1,1,2017,9,Skibbereen,South-West (IE),0,Tooreen
308456,"Dromig, Skibereen, Co Cork",NaN,Cork,285000.0,1,1,1,2,2021,9,Skibbereen,South-West (IE),0,Dromig
308457,"Rineen, Skibereen, Co. Cork",NaN,Cork,160000.0,1,1,1,1,2018,8,Skibbereen,South-West (IE),0,Rineen


In [4]:
irl.loc[irl['Address'].str.contains('Golf Links Rd, Youghal, Cork')]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
184360,"Delville Golf Links Rd, Youghal, Cork",NaN,Cork,215000.0,1,1,1,0,2017,12,Youghal,South-West (IE),0,Delville Golf Links Rd
190805,"Golf Links Rd, Youghal, Cork",NaN,Cork,350000.0,1,1,1,2,2021,9,Youghal,South-West (IE),0,Golf Links Rd


In [5]:
irl.dropna(subset=['Neighbourhood'], how='all', inplace=True)

In [6]:
irl.loc[irl['Address'].str.contains("1. St. Ciallian's View"), 'Neighbourhood'] = "St. Ciallian's View"
#final.loc[final['Address'].str.contains("Dun Laoghaire") & final['Town'].str.contains("Dublin"), 'Town'] = 'Dún Laoghaire'

In [7]:
irl.loc[irl['Address'].str.contains("Summerhaven") & irl['County'].str.contains("Leitrim"), 'Neighbourhood'] = 'Summerhill'

In [8]:
irl.loc[irl['Address'].str.contains("Block B") & irl['County'].str.contains("Leitrim"), 'Neighbourhood'] = 'Leitrim Marina'

In [9]:
irl.loc[irl['Neighbourhood'] == 'Main St', 'Neighbourhood'] = 'Main Street'

In [10]:
irl.loc[irl['Neighbourhood'].str.contains("Rd") == True]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
226,"Apartment 37 Block D, Clare Village, Clare Hal...",NaN,Dublin,169950.0,1,0,0,1,2010,1,Donaghmede,Dublin,1,Malahide Rd
498,"Boughlone House Mountrath Rd, Portlaoise",NaN,Laois,80000.0,0,1,1,1,2010,1,Portlaoise,Midlands,0,Boughlone House Mountrath Rd
539,"35 Colpe Park, Deepforde, Dublin Rd, Drogheda",NaN,Louth,272000.0,1,1,1,1,2010,1,Drogheda Louth,Border,0,Dublin Rd
628,"16 Enniskerry Rd, Phibsborough",Dublin 7,Dublin,190000.0,1,1,1,1,2010,1,Phibsborough,Dublin,0,Enniskerry Rd
1140,"23 Mcdermotts Tce, Carrick Rd, Dundalk",NaN,Louth,228000.0,1,1,1,1,2010,2,Dundalk,Border,0,Carrick Rd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308270,"8 Royal Oak Rd, Bagenalstown, Co Carlow",NaN,Carlow,125000.0,1,1,1,2,2015,8,Muine Bheag,South East (IE),0,Royal Oak Rd
308286,"12 Royal Oak Rd, Bagenalstown, Carlow",NaN,Carlow,120000.0,1,1,1,0,2012,2,Muine Bheag,South East (IE),0,Royal Oak Rd
308315,"Royal Oak Rd, Bagenalstown, Carlow",NaN,Carlow,60000.0,1,1,1,2,2016,3,Muine Bheag,South East (IE),0,Royal Oak Rd
308325,"25 Royal Oak Rd, Bagenalstown, Carlow",NaN,Carlow,141000.0,1,1,1,2,2016,12,Muine Bheag,South East (IE),0,Royal Oak Rd


In [11]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[-2:] == "Rd":
        str1 = str1[:-2] + " Road"
        v.append(str1)
    else:
        v.append(str1)

In [12]:
len(v)

306902

In [13]:
irl['Neighbourhood'] = v

In [14]:
irl.loc[irl['Address'] == "Boughlone House Mountrath Rd, Portlaoise"]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
498,"Boughlone House Mountrath Rd, Portlaoise",NaN,Laois,80000.0,0,1,1,1,2010,1,Portlaoise,Midlands,0,Boughlone House Mountrath Road


In [15]:
irl.drop(irl[irl['Address'] == "Frenchpark, Castlerea"].index, inplace=True)

In [16]:
irl['Neighbourhood'].nunique()

64545

In [17]:
#irl['Neighbourhood'] = v

In [18]:
irl['Neighbourhood'].nunique()

64545

In [19]:
irl.loc[irl['Address'].str.contains("Jamestown Apt") & irl['County'].str.contains("Leitrim"), 'Neighbourhood'] = 'Jamestown'

In [20]:
irl.loc[irl['Address'].str.contains("Cloone Garda Station") & irl['County'].str.contains("Leitrim"), 'Neighbourhood'] = 'Cloone'

In [21]:
irl.loc[irl['Address'].str.contains("Glenboy Post Office") & irl['County'].str.contains("Leitrim"), 'Neighbourhood'] = 'Glenboy'

In [22]:
irl[irl['Neighbourhood'].str.contains('St')].tail(30)

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
308312,"22 Kilcarrig St, Bagenalstown, Carlow",NaN,Carlow,85000.00,1,1,1,2,2015,12,Muine Bheag,South East (IE),0,Kilcarrig St
308313,"1 Church St, Bagenalstown, Carlow",NaN,Carlow,25000.00,1,1,1,1,2015,12,Muine Bheag,South East (IE),0,Church St
308321,"23 Kilcarrig St, Bagenalstown, Carlow",NaN,Carlow,75000.00,1,1,1,1,2016,10,Muine Bheag,South East (IE),0,Kilcarrig St
308332,"30 St Brigids Crescent, Bagenalstown, Carlow",NaN,Carlow,160000.00,1,1,1,1,2017,7,Muine Bheag,South East (IE),0,St Brigids Crescent
308334,"Stationhouse Rd, Bagenalstown, Carlow",NaN,Carlow,456000.00,1,1,1,2,2017,10,Muine Bheag,South East (IE),0,Stationhouse Road
308338,"14 Barret St, Bagenalstown, Carlow",NaN,Carlow,115000.00,1,1,1,2,2017,12,Muine Bheag,South East (IE),0,Barret St
308339,"19 Barrett St, Bagenalstown, Carlow",NaN,Carlow,80000.00,1,1,1,1,2018,1,Muine Bheag,South East (IE),0,Barrett St
308340,"5 Barrett Street, Bagenalstown, Carlow",NaN,Carlow,92000.00,1,1,1,2,2018,2,Muine Bheag,South East (IE),0,Barrett Street
308342,"35 St Brigids Crescent, Bagenalstown, Carlow",NaN,Carlow,135000.00,0,1,1,1,2018,6,Muine Bheag,South East (IE),0,St Brigids Crescent
308343,"14 St Brigids Crescent, Bagenalstown, Carlow",NaN,Carlow,90000.00,1,1,1,1,2018,8,Muine Bheag,South East (IE),0,St Brigids Crescent


In [23]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[-2:] == "St":
        str1 = str1[:-2] + " Street"
        v.append(str1)
    else:
        v.append(str1)

In [24]:
irl['Neighbourhood'] = v

In [25]:
irl[irl['Neighbourhood'].str.contains('St')].tail(30)

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
308312,"22 Kilcarrig St, Bagenalstown, Carlow",NaN,Carlow,85000.00,1,1,1,2,2015,12,Muine Bheag,South East (IE),0,Kilcarrig Street
308313,"1 Church St, Bagenalstown, Carlow",NaN,Carlow,25000.00,1,1,1,1,2015,12,Muine Bheag,South East (IE),0,Church Street
308321,"23 Kilcarrig St, Bagenalstown, Carlow",NaN,Carlow,75000.00,1,1,1,1,2016,10,Muine Bheag,South East (IE),0,Kilcarrig Street
308332,"30 St Brigids Crescent, Bagenalstown, Carlow",NaN,Carlow,160000.00,1,1,1,1,2017,7,Muine Bheag,South East (IE),0,St Brigids Crescent
308334,"Stationhouse Rd, Bagenalstown, Carlow",NaN,Carlow,456000.00,1,1,1,2,2017,10,Muine Bheag,South East (IE),0,Stationhouse Road
308338,"14 Barret St, Bagenalstown, Carlow",NaN,Carlow,115000.00,1,1,1,2,2017,12,Muine Bheag,South East (IE),0,Barret Street
308339,"19 Barrett St, Bagenalstown, Carlow",NaN,Carlow,80000.00,1,1,1,1,2018,1,Muine Bheag,South East (IE),0,Barrett Street
308340,"5 Barrett Street, Bagenalstown, Carlow",NaN,Carlow,92000.00,1,1,1,2,2018,2,Muine Bheag,South East (IE),0,Barrett Street
308342,"35 St Brigids Crescent, Bagenalstown, Carlow",NaN,Carlow,135000.00,0,1,1,1,2018,6,Muine Bheag,South East (IE),0,St Brigids Crescent
308343,"14 St Brigids Crescent, Bagenalstown, Carlow",NaN,Carlow,90000.00,1,1,1,1,2018,8,Muine Bheag,South East (IE),0,St Brigids Crescent


In [26]:
irl[irl['Neighbourhood'].str.contains('Rd.')]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
2849,"99 Knockaphunta Park, Westport Rd., Castlebar",NaN,Mayo,185000.0,1,1,1,1,2010,3,Castlebar,West,0,Westport Rd.
2888,"5 The Paddocks, Bridge Rd., Listowel",NaN,Kerry,200000.0,1,1,1,1,2010,3,Listowel,South-West (IE),0,Bridge Rd.
3939,"177 Dun Eoin, Ballinrea Rd., Carrigaline",NaN,Cork,206000.0,1,1,1,1,2010,4,Carrigaline,South-West (IE),0,Ballinrea Rd.
4273,"35 Greenoaks, Rockshire Rd., Ferrybank",NaN,Kilkenny,175500.0,1,1,1,1,2010,5,Ferrybank Kilkenny,South East (IE),0,Rockshire Rd.
5220,"Rockhaven, Camden Rd., Crosshaven",NaN,Cork,320000.0,1,1,1,2,2010,6,Crosshaven,South-West (IE),0,Camden Rd.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307175,"29 Woodlands Rd Dun Laoghaire, Dun Laoghaire, ...",NaN,Dublin,430000.0,1,1,1,1,2014,7,Dún Laoghaire,Dublin,0,Woodlands Rd Dun Laoghaire
307212,"22 Eden Rd Upper, Dun Laoghaire, Dublin",NaN,Dublin,580000.0,1,1,1,1,2015,3,Dún Laoghaire,Dublin,0,Eden Rd Upper
307213,"54 Mounttown Rd Lower, Dun Laoghaire, Dublin",NaN,Dublin,490000.0,1,1,1,1,2015,3,Dún Laoghaire,Dublin,0,Mounttown Rd Lower
307429,"5 Joyce Rd Lusk Village, Lusk Village, Dublin",NaN,Dublin,186000.0,1,1,1,1,2018,4,Lusk,Dublin,0,Joyce Rd Lusk Village


In [27]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[-3:] == "Rd.":
        str1 = str1[:-3] + " Road"
        v.append(str1)
    else:
        v.append(str1)

In [28]:
irl['Neighbourhood'] = v

In [29]:
# will use this later
def lastnum(string):
    s = string[::-1]
    m = re.search('\d', s)
    pos = len(s) - m.start(0)
    return pos

In [30]:
#df11
df11 = irl.loc[irl['Neighbourhood'] == 'Waterford']
ne = np.array(df11['Address'])
updated = []
for i in range(len(ne)):
    neighbour = ne[i].split(",")
    #print(neighbour)
    #print(neighbour[0])
    n = neighbour[0]
    if n[0].isnumeric():
        dig = lastnum(n)
        updated.append(n[dig:].strip())
    else:
        updated.append(n.strip())

In [31]:
df11add = df11['Address']
df11add = list(df11add)

In [32]:
irl.drop(irl[irl['Address'].isin(df11add)].index, inplace=True)

In [33]:
#updated
df11['Neighbourhood'] = updated


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:
irl = pd.concat([irl, df11], ignore_index=True)

In [35]:
l = list(irl['Town'].unique())
fix_neigh = np.array(irl['Neighbourhood'])
change = []
for i in range(len(fix_neigh)):
    if fix_neigh[i] in l and fix_neigh[i] not in change:
        change.append(fix_neigh[i])
    else:
        continue

In [36]:
double_name = irl.loc[irl['Neighbourhood'].isin(change)]
double_names = np.array(double_name['Neighbourhood'])
double_town = np.array(double_name['Town'])
double = []
for i in range(len(double_names)):
    if double_town[i] == double_names[i]:
        add = np.array(double_name['Address'])
        adds = add[i].split(",")
        if len(adds) >= 3:
            double.append(adds[-3].strip())
        else:
            double.append(adds[-2].strip())
    else:
        double.append(double_names[i].strip())

In [37]:
irl.loc[irl['Address'].str.contains('Golf Links Rd, Youghal, Cork')]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
183467,"Delville Golf Links Rd, Youghal, Cork",NaN,Cork,215000.0,1,1,1,0,2017,12,Youghal,South-West (IE),0,Delville Golf Links Road
189888,"Golf Links Rd, Youghal, Cork",NaN,Cork,350000.0,1,1,1,2,2021,9,Youghal,South-West (IE),0,Golf Links Road


In [38]:
double_name['Neighbourhood'] = double


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [39]:
doub = double_name['Address']
doub_list = list(doub)

In [40]:
irl.drop(irl[irl['Address'].isin(doub)].index, inplace=True)

In [41]:
irl = pd.concat([irl, double_name], ignore_index=True)

In [42]:
irl.loc[irl['Address'].str.contains('Golf Links Rd, Youghal, Cork')]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
169895,"Delville Golf Links Rd, Youghal, Cork",NaN,Cork,215000.0,1,1,1,0,2017,12,Youghal,South-West (IE),0,Delville Golf Links Road
175960,"Golf Links Rd, Youghal, Cork",NaN,Cork,350000.0,1,1,1,2,2021,9,Youghal,South-West (IE),0,Golf Links Road


In [43]:
irl.loc[irl['Neighbourhood'].str.contains("Avenue") & irl['Town'].str.contains("Dromahair"), 'Neighbourhood'] = 'Woodlands Avenue'

In [44]:
irl.loc[irl['Neighbourhood'].str.contains("St") == True].head(10)

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
105,"14 St. Aonghus Road, Tallaght",Dublin 24,Dublin,220000.0,1,1,1,0,2010,1,Tallaght,Dublin,0,St. Aonghus Road
118,"32 Connolly Street, Arklow",NaN,Wicklow,140000.0,1,1,1,2,2010,1,Arklow,Mid-East,0,Connolly Street
148,"Tirconnell Street, Ballyshannon",NaN,Donegal,197000.0,1,1,1,1,2010,1,Ballyshannon,Border,0,Tirconnell Street
163,"33, St. Patrick's Estate, Ballina",NaN,Mayo,127000.0,1,1,1,1,2010,1,Ballina Mayo,West,0,St. Patrick's Estate
176,"Stradbally, Kilcolgan",NaN,Galway,340000.0,1,1,1,1,2010,1,Kilcolgan,West,0,Stradbally
189,"31 St Brigids Place, Portlaoise",NaN,Laois,70000.0,0,1,1,1,2010,1,Portlaoise,Midlands,0,St Brigids Place
195,"Glendower, Old Station Road, Portmarnock",NaN,Dublin,1475000.0,1,1,1,1,2010,1,Portmarnock,Dublin,0,Old Station Road
212,"5 Ross Street, Stoneybatter",NaN,Dublin,190000.0,1,1,1,1,2010,1,Stoneybatter,Dublin,0,Ross Street
233,"22 4 Herberton Street, Rialto",Dublin 8,Dublin,182000.0,1,0,0,1,2010,1,Rialto,Dublin,0,Herberton Street
272,"16 St. Enda's Drive, Rathfarnham",Dublin 16,Dublin,390000.0,1,1,1,1,2010,1,Rathfarnham,Dublin,0,St. Enda's Drive


In [45]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[:3] == "St ":
        str1 = "St. " + str1[3:]
        v.append(str1)
    else:
        v.append(str1)

In [46]:
irl['Neighbourhood'] = v

In [47]:
irl.loc[irl['Neighbourhood'].str.contains("St") == True].head(10)

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
105,"14 St. Aonghus Road, Tallaght",Dublin 24,Dublin,220000.0,1,1,1,0,2010,1,Tallaght,Dublin,0,St. Aonghus Road
118,"32 Connolly Street, Arklow",NaN,Wicklow,140000.0,1,1,1,2,2010,1,Arklow,Mid-East,0,Connolly Street
148,"Tirconnell Street, Ballyshannon",NaN,Donegal,197000.0,1,1,1,1,2010,1,Ballyshannon,Border,0,Tirconnell Street
163,"33, St. Patrick's Estate, Ballina",NaN,Mayo,127000.0,1,1,1,1,2010,1,Ballina Mayo,West,0,St. Patrick's Estate
176,"Stradbally, Kilcolgan",NaN,Galway,340000.0,1,1,1,1,2010,1,Kilcolgan,West,0,Stradbally
189,"31 St Brigids Place, Portlaoise",NaN,Laois,70000.0,0,1,1,1,2010,1,Portlaoise,Midlands,0,St. Brigids Place
195,"Glendower, Old Station Road, Portmarnock",NaN,Dublin,1475000.0,1,1,1,1,2010,1,Portmarnock,Dublin,0,Old Station Road
212,"5 Ross Street, Stoneybatter",NaN,Dublin,190000.0,1,1,1,1,2010,1,Stoneybatter,Dublin,0,Ross Street
233,"22 4 Herberton Street, Rialto",Dublin 8,Dublin,182000.0,1,0,0,1,2010,1,Rialto,Dublin,0,Herberton Street
272,"16 St. Enda's Drive, Rathfarnham",Dublin 16,Dublin,390000.0,1,1,1,1,2010,1,Rathfarnham,Dublin,0,St. Enda's Drive


In [48]:
irl['Neighbourhood'].nunique()

67468

In [49]:
connacht = ['Roscommon', 'Leitrim', 'Sligo', 'Mayo', 'Galway']
ulster = ['Donegal', 'Cavan', 'Monaghan']
munster = ['Kerry', 'Cork', 'Tipperary', 'Clare', 'Limerick', 'Waterford']
prov = []
z = np.array(irl['County'])
for i in range(len(z)):
    if z[i] in connacht:
        prov.append('Connacht')
    elif z[i] in ulster:
        prov.append('Ulster')
    elif z[i] in munster:
        prov.append('Munster')
    else:
        prov.append('Leinster')

In [50]:
irl['Province'] = prov

In [51]:
irl

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province
0,"5 Braemor Drive, Churchtown",NaN,Dublin,343000.0,1,1,1,0,2010,1,Churchtown Dublin,Dublin,0,Braemor Drive,Leinster
1,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,185000.0,1,0,0,1,2010,1,Portlaoise,Midlands,0,Summerhill Lane,Leinster
2,"1 Meadow Avenue, Dundrum",Dublin 14,Dublin,438500.0,1,1,1,1,2010,1,Dundrum Dublin,Dublin,0,Meadow Avenue,Leinster
3,"1 The Haven, Mornington",NaN,Meath,400000.0,1,1,1,1,2010,1,Mornington,Mid-East,0,The Haven,Leinster
4,"12 Sallymount Avenue, Ranelagh",NaN,Dublin,425000.0,1,1,1,1,2010,1,Ranelagh,Dublin,0,Sallymount Avenue,Leinster
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306896,"Donore, Bagenalstown, Co. Carlow",NaN,Carlow,148000.0,1,1,1,1,2013,4,Muine Bheag,South East (IE),0,Donore,Leinster
306897,"Royal Oak, Bagenalstown, County Carlow",NaN,Carlow,95000.0,1,1,1,2,2020,5,Muine Bheag,South East (IE),0,Royal Oak,Leinster
306898,"Loughlinstown, Rathoath, Co Meath",NaN,Meath,610000.0,1,0,0,2,2013,5,Ratoath,Mid-East,0,Loughlinstown,Leinster
306899,"5 Streamstown, Rathoath, Co Meath",NaN,Meath,380000.0,1,1,1,1,2017,12,Ratoath,Mid-East,0,Streamstown,Leinster


In [52]:
commas1 = np.array(irl['Address'])
v = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[:1].isnumeric():
        v.append('Estate')
    else:
        v.append('Other')

In [53]:
irl['Estate'] = v

In [54]:
#irl.head(50)

In [55]:
irl.loc[irl['Neighbourhood'].str.contains("O Connel") == True].head(20)

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
77000,"The Manor, O Connell Square, Edenderry",NaN,Offaly,90000.0,1,1,1,1,2015,9,Edenderry,Midlands,0,O Connell Square,Leinster,Other
84103,"Mountain View House, O Connell St, Dungarvan",NaN,Waterford,390000.0,1,1,1,2,2016,3,Dungarvan Waterford,South East (IE),0,O Connell Street,Munster,Other
86460,"11 Doirebeag, O Connell St, Dungarvan",NaN,Waterford,147000.0,1,1,1,0,2016,6,Dungarvan Waterford,South East (IE),0,O Connell Street,Munster,Estate
94067,"Apt 8 O Connell Court, O Connell Street, Water...",NaN,Waterford,75000.0,1,1,1,1,2016,12,Waterford City,South East (IE),1,O Connell Street,Munster,Other
94184,"Apt 25 O Connell Court, O Connell Street, Wate...",NaN,Waterford,77000.0,1,1,1,1,2016,12,Waterford City,South East (IE),1,O Connell Street,Munster,Other
102011,"Beech Cottage, O Connell St, Dungarvan",NaN,Waterford,74000.0,0,1,1,1,2017,7,Dungarvan Waterford,South East (IE),0,O Connell Street,Munster,Other
109463,"Apt 1 3 Hillside Terrace, O Connell Place, Fermoy",NaN,Cork,179050.0,1,1,1,0,2018,1,Fermoy,South-West (IE),1,O Connell Place,Munster,Other
140491,"Apt 2, 71 O Connell St, Ennis",NaN,Clare,108142.0,1,1,1,1,2020,1,Ennis,Mid-West,1,O Connell Street,Munster,Other
146003,"First Floor Apartment, 59 O Connell Street, Du...",NaN,Waterford,160000.0,1,1,1,2,2020,8,Dungarvan Waterford,South East (IE),1,O Connell Street,Munster,Other
150521,"Cul Na Greine, Higher O Connell Street, Kinsale",NaN,Cork,595000.0,1,1,1,2,2020,11,Kinsale,South-West (IE),0,Higher O Connell Street,Munster,Other


In [56]:
irl.loc[irl['Neighbourhood'].str.contains("11O'Connell Gardens") & irl['Town'].str.contains("Sandymount"), 'Neighbourhood'] = "O'Connell Gardens"

In [57]:
irl.loc[irl['Neighbourhood'].str.contains("O Connel") == True].head(20)

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
77000,"The Manor, O Connell Square, Edenderry",NaN,Offaly,90000.0,1,1,1,1,2015,9,Edenderry,Midlands,0,O Connell Square,Leinster,Other
84103,"Mountain View House, O Connell St, Dungarvan",NaN,Waterford,390000.0,1,1,1,2,2016,3,Dungarvan Waterford,South East (IE),0,O Connell Street,Munster,Other
86460,"11 Doirebeag, O Connell St, Dungarvan",NaN,Waterford,147000.0,1,1,1,0,2016,6,Dungarvan Waterford,South East (IE),0,O Connell Street,Munster,Estate
94067,"Apt 8 O Connell Court, O Connell Street, Water...",NaN,Waterford,75000.0,1,1,1,1,2016,12,Waterford City,South East (IE),1,O Connell Street,Munster,Other
94184,"Apt 25 O Connell Court, O Connell Street, Wate...",NaN,Waterford,77000.0,1,1,1,1,2016,12,Waterford City,South East (IE),1,O Connell Street,Munster,Other
102011,"Beech Cottage, O Connell St, Dungarvan",NaN,Waterford,74000.0,0,1,1,1,2017,7,Dungarvan Waterford,South East (IE),0,O Connell Street,Munster,Other
109463,"Apt 1 3 Hillside Terrace, O Connell Place, Fermoy",NaN,Cork,179050.0,1,1,1,0,2018,1,Fermoy,South-West (IE),1,O Connell Place,Munster,Other
140491,"Apt 2, 71 O Connell St, Ennis",NaN,Clare,108142.0,1,1,1,1,2020,1,Ennis,Mid-West,1,O Connell Street,Munster,Other
146003,"First Floor Apartment, 59 O Connell Street, Du...",NaN,Waterford,160000.0,1,1,1,2,2020,8,Dungarvan Waterford,South East (IE),1,O Connell Street,Munster,Other
150521,"Cul Na Greine, Higher O Connell Street, Kinsale",NaN,Cork,595000.0,1,1,1,2,2020,11,Kinsale,South-West (IE),0,Higher O Connell Street,Munster,Other


In [58]:
irl.loc[irl['Neighbourhood'].str.contains("O Connell") & irl['Town'].str.contains("Waterford"), 'Neighbourhood'] = "O'Connell Street"

In [59]:
irl.loc[irl['Neighbourhood'].str.contains("O Connel") == True].head(20)

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
77000,"The Manor, O Connell Square, Edenderry",NaN,Offaly,90000.0,1,1,1,1,2015,9,Edenderry,Midlands,0,O Connell Square,Leinster,Other
109463,"Apt 1 3 Hillside Terrace, O Connell Place, Fermoy",NaN,Cork,179050.0,1,1,1,0,2018,1,Fermoy,South-West (IE),1,O Connell Place,Munster,Other
140491,"Apt 2, 71 O Connell St, Ennis",NaN,Clare,108142.0,1,1,1,1,2020,1,Ennis,Mid-West,1,O Connell Street,Munster,Other
150521,"Cul Na Greine, Higher O Connell Street, Kinsale",NaN,Cork,595000.0,1,1,1,2,2020,11,Kinsale,South-West (IE),0,Higher O Connell Street,Munster,Other
168318,"63 Lower O Connell St, Kinsale, Cork",NaN,Cork,300000.0,1,1,1,1,2017,2,Kinsale,South-West (IE),0,Lower O Connell Street,Munster,Estate
170030,"Lower O Connell St, Kinsale, Cork",NaN,Cork,617500.0,1,1,1,2,2018,1,Kinsale,South-West (IE),0,Lower O Connell Street,Munster,Other
181576,"1 Old Wellington Terace, O Connel Avenue, Lime...",NaN,Limerick,165000.0,1,1,1,1,2014,7,Limerick,Mid-West,0,O Connel Avenue,Munster,Estate
182332,"Ascot House, O Connell Ave, Limerick",NaN,Limerick,230000.0,1,1,1,1,2015,3,Limerick,Mid-West,0,O Connell Ave,Munster,Other
182425,"Portarthur, O Connell Ave, Limerick",NaN,Limerick,342500.0,1,1,1,1,2015,4,Limerick,Mid-West,0,O Connell Ave,Munster,Other
184327,"2 Verona Esplanade, O Connell Ave, Limerick",NaN,Limerick,209500.0,1,1,1,1,2017,5,Limerick,Mid-West,0,O Connell Ave,Munster,Estate


In [60]:
irl.loc[irl['Neighbourhood'].str.contains("O Connell") & irl['Town'].str.contains("Edenderry"), 'Neighbourhood'] = "O'Connell Square"

In [61]:
irl.loc[irl['Neighbourhood'].str.contains("O Connel") == True].head(20)

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
109463,"Apt 1 3 Hillside Terrace, O Connell Place, Fermoy",NaN,Cork,179050.0,1,1,1,0,2018,1,Fermoy,South-West (IE),1,O Connell Place,Munster,Other
140491,"Apt 2, 71 O Connell St, Ennis",NaN,Clare,108142.0,1,1,1,1,2020,1,Ennis,Mid-West,1,O Connell Street,Munster,Other
150521,"Cul Na Greine, Higher O Connell Street, Kinsale",NaN,Cork,595000.0,1,1,1,2,2020,11,Kinsale,South-West (IE),0,Higher O Connell Street,Munster,Other
168318,"63 Lower O Connell St, Kinsale, Cork",NaN,Cork,300000.0,1,1,1,1,2017,2,Kinsale,South-West (IE),0,Lower O Connell Street,Munster,Estate
170030,"Lower O Connell St, Kinsale, Cork",NaN,Cork,617500.0,1,1,1,2,2018,1,Kinsale,South-West (IE),0,Lower O Connell Street,Munster,Other
181576,"1 Old Wellington Terace, O Connel Avenue, Lime...",NaN,Limerick,165000.0,1,1,1,1,2014,7,Limerick,Mid-West,0,O Connel Avenue,Munster,Estate
182332,"Ascot House, O Connell Ave, Limerick",NaN,Limerick,230000.0,1,1,1,1,2015,3,Limerick,Mid-West,0,O Connell Ave,Munster,Other
182425,"Portarthur, O Connell Ave, Limerick",NaN,Limerick,342500.0,1,1,1,1,2015,4,Limerick,Mid-West,0,O Connell Ave,Munster,Other
184327,"2 Verona Esplanade, O Connell Ave, Limerick",NaN,Limerick,209500.0,1,1,1,1,2017,5,Limerick,Mid-West,0,O Connell Ave,Munster,Estate
187337,"7 Verona Espanade, O Connell Ave, Limerick",NaN,Limerick,325000.0,1,1,1,1,2020,10,Limerick,Mid-West,0,O Connell Ave,Munster,Estate


In [62]:
irl.loc[irl['Neighbourhood'].str.contains("O Connell") & irl['Town'].str.contains("Westmeath"), 'Neighbourhood'] = "O'Connell Street"
irl.loc[irl['Neighbourhood'].str.contains("O Connell") & irl['Town'].str.contains("Ennis"), 'Neighbourhood'] = "O'Connell Street"
irl.loc[irl['Neighbourhood'].str.contains("O Connell") & irl['Town'].str.contains("Ennis"), 'Neighbourhood'] = "O'Connell Place"
irl.loc[irl['Neighbourhood'].str.contains("O Connell") & irl['Town'].str.contains("Arklow"), 'Neighbourhood'] = "O'Connell Terrace"

In [63]:
irl.loc[irl['Neighbourhood'].str.contains("O Connel") == True].head(20)

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
109463,"Apt 1 3 Hillside Terrace, O Connell Place, Fermoy",NaN,Cork,179050.0,1,1,1,0,2018,1,Fermoy,South-West (IE),1,O Connell Place,Munster,Other
150521,"Cul Na Greine, Higher O Connell Street, Kinsale",NaN,Cork,595000.0,1,1,1,2,2020,11,Kinsale,South-West (IE),0,Higher O Connell Street,Munster,Other
168318,"63 Lower O Connell St, Kinsale, Cork",NaN,Cork,300000.0,1,1,1,1,2017,2,Kinsale,South-West (IE),0,Lower O Connell Street,Munster,Estate
170030,"Lower O Connell St, Kinsale, Cork",NaN,Cork,617500.0,1,1,1,2,2018,1,Kinsale,South-West (IE),0,Lower O Connell Street,Munster,Other
181576,"1 Old Wellington Terace, O Connel Avenue, Lime...",NaN,Limerick,165000.0,1,1,1,1,2014,7,Limerick,Mid-West,0,O Connel Avenue,Munster,Estate
182332,"Ascot House, O Connell Ave, Limerick",NaN,Limerick,230000.0,1,1,1,1,2015,3,Limerick,Mid-West,0,O Connell Ave,Munster,Other
182425,"Portarthur, O Connell Ave, Limerick",NaN,Limerick,342500.0,1,1,1,1,2015,4,Limerick,Mid-West,0,O Connell Ave,Munster,Other
184327,"2 Verona Esplanade, O Connell Ave, Limerick",NaN,Limerick,209500.0,1,1,1,1,2017,5,Limerick,Mid-West,0,O Connell Ave,Munster,Estate
187337,"7 Verona Espanade, O Connell Ave, Limerick",NaN,Limerick,325000.0,1,1,1,1,2020,10,Limerick,Mid-West,0,O Connell Ave,Munster,Estate
187720,"2 Munster Terrace, O Connell Ave, Limerick",NaN,Limerick,365000.0,1,1,1,2,2021,3,Limerick,Mid-West,0,O Connell Ave,Munster,Estate


In [64]:
irl.loc[irl['Neighbourhood'].str.contains("O Connell") & irl['Town'].str.contains("Sandymount"), 'Neighbourhood'] = "O'Connell Gardens Bath Ave"
irl.loc[irl['Neighbourhood'].str.contains("O Connell") & irl['Town'].str.contains("Listowel"), 'Neighbourhood'] = "O'Connell's Ave"
irl.loc[irl['Neighbourhood'].str.contains("O Connell") & irl['Town'].str.contains("Fermoy"), 'Neighbourhood'] = "O'Connell Place"

In [65]:
irl.loc[irl['Neighbourhood'].str.contains("Ave")]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
2,"1 Meadow Avenue, Dundrum",Dublin 14,Dublin,438500.00,1,1,1,1,2010,1,Dundrum Dublin,Dublin,0,Meadow Avenue,Leinster,Estate
4,"12 Sallymount Avenue, Ranelagh",NaN,Dublin,425000.00,1,1,1,1,2010,1,Ranelagh,Dublin,0,Sallymount Avenue,Leinster,Estate
7,"206 Philipsburgh Avenue, Marino",Dublin 3,Dublin,430000.00,1,1,1,1,2010,1,Marino,Dublin,0,Philipsburgh Avenue,Leinster,Estate
17,"54., Brides Glen Avenue, Swords",NaN,Dublin,337500.00,1,1,1,2,2010,1,Swords,Dublin,0,Brides Glen Avenue,Leinster,Estate
82,"121 Medebawn, Long Avenue, Dundalk",NaN,Louth,237885.46,1,0,0,2,2010,1,Dundalk,Border,0,Long Avenue,Leinster,Estate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305978,"7 Cromwellsfort Ave, Wexford, Wexford",NaN,Wexford,197000.00,1,1,1,1,2019,10,Wexford,South East (IE),0,7 Cromwellsfort Ave,Leinster,Estate
306007,"44 Cromwellsfort Ave, Wexford, Wexford",NaN,Wexford,165000.00,1,1,1,1,2020,3,Wexford,South East (IE),0,44 Cromwellsfort Ave,Leinster,Estate
306216,"141 Mount Carmel Ave, Wicklow, Wicklow",NaN,Wicklow,179000.00,1,1,1,1,2017,2,Wicklow,Mid-East,0,141 Mount Carmel Ave,Leinster,Estate
306221,"18 St Patricks Ave, Wicklow, Wicklow",NaN,Wicklow,220000.00,1,1,1,1,2017,5,Wicklow,Mid-East,0,18 St Patricks Ave,Leinster,Estate


In [66]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[-3:] == "Ave" or str1[-3:] == "ave":
        v.append(str1)
    else:
        continue

In [67]:
len(v)

3281

In [68]:
#v

In [69]:
commas1 = np.array(irl['Neighbourhood'])
ave = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1 in v:
        str2 = str1[:-4] + " Avenue"
        ave.append(str2)
    else:
        ave.append(str1)

In [70]:
irl['Neighbourhood'] = ave

In [71]:
commas1 = np.array(irl['Neighbourhood'])
sq = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[-2:] == 'Sq' or str1[-2:] == 'sq':
        str2 = str1[:-3] + " Square"
        sq.append(str2)
    else:
        sq.append(str1)

In [72]:
irl['Neighbourhood'] = sq

In [73]:
commas1 = np.array(irl['Neighbourhood'])
stz = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[-2:] == 'St' or str1[-2:] == 'st':
        str2 = str1[:-3] + " Street"
        stz.append(str2)
    else:
        stz.append(str1)

In [74]:
irl['Neighbourhood'] = stz

In [75]:
commas1 = np.array(irl['Neighbourhood'])
dr = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[-2:] == 'Dr'or str1[-2:] == 'dr':
        str2 = str1[:-3] + " Drive"
        dr.append(str2)
    else:
        dr.append(str1)

In [76]:
irl['Neighbourhood'] = dr

In [77]:
irl.loc[irl['Address'].str.contains("Lowergate Sq")]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
106841,"Apt 5, Lowergate Square, Cashel",NaN,Tipperary,49000.0,1,1,1,2,2017,11,Cashel,South East (IE),1,Lowergate Square,Munster,Other
127602,"Gas House Lane, Lowergate Sq, Cashel",NaN,Tipperary,197000.0,1,1,1,1,2019,4,Cashel,South East (IE),0,Lowergate Square,Munster,Other


In [78]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[-3:] == "Ave":
        v.append(str1)
    else:
        continue

In [79]:
v

[]

In [80]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

In [81]:
commas1 = np.array(irl['Neighbourhood'])
no = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if has_numbers(str1):
        no.append(str1)
    else:
        continue

In [82]:
len(no)

4345

In [83]:
#no

In [84]:
b = set(no)

In [85]:
len(b)

4200

In [86]:
#b

In [87]:
irl.loc[irl['Neighbourhood'].str.contains("Unit 229 Roseberry Hill"), 'Neighbourhood'] = 'Roseberry Hill'

In [88]:
irl.loc[irl['Neighbourhood'].str.contains("Oakwood") & irl['Town'].str.contains("Killarney"), 'Neighbourhood'] = 'Oakwood'

In [89]:
irl.loc[irl['Neighbourhood'].str.contains("Unit 1 Rosemount") & irl['Town'].str.contains("Roscrea"), 'Neighbourhood'] = 'Rosemount'

In [90]:
irl.loc[irl['Neighbourhood'].str.contains("Clanbrassil  Street"), 'Neighbourhood'] = 'Clanbrassil Street'

In [91]:
irl.loc[irl['Neighbourhood'].str.contains("7/7A St Patrick's Terrace"), 'Neighbourhood'] = 'St. Patricks Terrace'

In [92]:
irl.loc[irl['Neighbourhood'].str.contains("Ocean View"), 'Neighbourhood'] = 'Ocean View'

In [93]:
irl.loc[irl['Neighbourhood'].str.contains("Ballsbridge Gardens"), 'Neighbourhood'] = 'Ballsbridge Gardens' 

In [94]:
irl.loc[irl['Neighbourhood'].str.contains("12-13 North Main Street"), 'Neighbourhood'] = 'North Main Street'

In [95]:
irl.loc[irl['Neighbourhood'].str.contains("Chancellery Park Road"), 'Neighbourhood'] = 'Chancellery Park Road'

In [96]:
irl.loc[irl['Neighbourhood'].str.contains("Frederick Court"), 'Neighbourhood'] = 'Frederick Court'

In [97]:
irl.loc[irl['Neighbourhood'].str.contains("Bree") & irl['County'].str.contains('Monaghan'), 'Neighbourhood'] = 'Bree'

In [98]:
irl.loc[irl['Neighbourhood'].str.contains("Industry Place"), 'Neighbourhood'] = 'Industry Place'

In [99]:
irl.loc[irl['Neighbourhood'].str.contains("Shandon Street") & irl['County'].str.contains('Cork'), 'Neighbourhood'] = 'Shandon Street'

In [100]:
irl.loc[irl['Neighbourhood'].str.contains("Tawnalary"), 'Neighbourhood'] = 'Tawnalary'

In [101]:
irl.loc[irl['Neighbourhood'].str.contains("Turners Hill"), 'Neighbourhood'] = 'Turners Hill'

In [102]:
irl.loc[irl['Neighbourhood'].str.contains("Spanish Cove Apts"), 'Neighbourhood'] = 'Spanish Cove Apts'

In [103]:
irl.loc[irl['Neighbourhood'].str.contains("Clearstream Court"), 'Neighbourhood'] = 'Clearstream Court' 

In [104]:
irl.loc[irl['Neighbourhood'].str.contains("Cashel") & irl['Town'].str.contains('Cashel'), 'Neighbourhood'] = 'Ballytarsna Hackett'

In [105]:
irl.loc[irl['Neighbourhood'].str.contains("Lowergate"), 'Neighbourhood'] = 'Lowergate'

In [106]:
irl.loc[irl['Neighbourhood'].str.contains("Watergate Street"), 'Neighbourhood'] = 'Watergate Street'

In [107]:
irl.loc[irl['Neighbourhood'].str.contains("St Anthony's"), 'Neighbourhood'] = "St. Anthony's"

In [108]:
irl.loc[irl['Neighbourhood'].str.contains("Mellifont Court"), 'Neighbourhood'] = 'Mellifont Court'

In [109]:
irl.loc[irl['Neighbourhood'].str.contains("Rampart View"), 'Neighbourhood'] = 'Rampart View'

In [110]:
irl.loc[irl['Neighbourhood'].str.contains("The Quadrants"), 'Neighbourhood'] = 'The Quadrants' 

In [111]:
irl.loc[irl['Neighbourhood'].str.contains("Bishop's Water"), 'Neighbourhood'] = "Bishop's Water"

In [112]:
irl.loc[irl['Neighbourhood'].str.contains("Friars Hill"), 'Neighbourhood'] = 'Friars Hill'

In [113]:
irl.loc[irl['Neighbourhood'].str.contains("31/33 Merrion Road"), 'Neighbourhood'] = 'Merrion Road'

In [114]:
irl.loc[irl['Neighbourhood'].str.contains("31-33 Merrion Road"), 'Neighbourhood'] = 'Merrion Road' 

In [115]:
irl.loc[irl['Neighbourhood'].str.contains("- 33 Merrion Road"), 'Neighbourhood'] = 'Merrion Road'

In [116]:
irl.drop(irl[irl['Neighbourhood'].str.contains("Block")].index, inplace=True)

In [117]:
irl.shape

(306626, 16)

In [118]:
commas1 = np.array(irl['Neighbourhood'])
no = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if has_numbers(str1):
        no.append(str1)
    else:
        continue

In [119]:
b = set(no)

In [120]:
len(b)

4052

In [121]:
# rm sites
irl.drop(irl[irl['Neighbourhood'].str.contains("Site")].index, inplace=True)
irl.drop(irl[irl['Neighbourhood'].str.contains("Lot")].index, inplace=True)

In [122]:
irl.loc[irl['Neighbourhood'].str.contains("Lot 15") & irl['County'].str.contains('Kildare'), 'Neighbourhood'] = 'Millennium Park'

In [123]:
irl.drop(irl[irl['Neighbourhood'].str.contains("Lot") & irl['Town'].str.contains('Cherrywood')].index, inplace=True)

In [124]:
irl.loc[irl['Neighbourhood'].str.contains("Lot") & irl['County'].str.contains('Dublin'), 'Neighbourhood'] = 'South Lotts Road'

In [125]:
irl.shape

(306502, 16)

In [126]:
commas1 = np.array(irl['Neighbourhood'])
no = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if has_numbers(str1):
        no.append(str1)
    else:
        continue

In [127]:
b = set(no)

In [128]:
len(b)

4029

In [129]:
#b

In [130]:
irl.loc[irl['Neighbourhood'].str.contains("Roseberry Hill"), 'Neighbourhood'] = 'Roseberry Hill'

In [131]:
irl.loc[irl['Neighbourhood'].str.contains("Baker's Court"), 'Neighbourhood'] = "Baker's Court" 

In [132]:
irl.loc[irl['Neighbourhood'].str.contains("Bakers Court"), 'Neighbourhood'] = "Baker's Court"

In [133]:
irl.loc[irl['Neighbourhood'].str.contains("51-53 Main  Street"), 'Neighbourhood'] = 'Main Street'

In [134]:
irl.loc[irl['Neighbourhood'].str.contains("51-53 Main Street"), 'Neighbourhood'] = 'Main Street'

In [135]:
irl.loc[irl['Neighbourhood'].str.contains("Parnell  Street"), 'Neighbourhood'] = 'Parnell Street'

In [136]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    if commas1[i] in b:
        str1 = commas1[i]
        dig = lastnum(str1)
        if str1[:4] == "Apt." or str1[:4] == "apt.":
            str2 = str1[dig:]
            v.append(str2)
        else:
            v.append(str1)
    else:
        v.append(commas1[i])

In [137]:
#v

In [138]:
irl['Neighbourhood'] = v

In [139]:
#irl.loc[irl['Address'].str.contains("Tuam Shopping Centre")]

In [140]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    if commas1[i] in b:
        str1 = commas1[i]
        dig = lastnum(str1)
        if str1[:5] == "House":
            str2 = str1[dig:]
            v.append(str2.strip())
        else:
            v.append(str1.strip())
    else:
        v.append(commas1[i])

In [141]:
#v

In [142]:
irl['Neighbourhood'] = v

In [143]:
#irl.loc[irl['Address'].str.contains("Church Avenue")]

In [144]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    if commas1[i] in b:
        str1 = commas1[i]
        dig = lastnum(str1)
        if str1[:3] == "No.":
            str2 = str1[dig:]
            v.append(str2.strip())
        else:
            v.append(str1.strip())
    else:
        v.append(commas1[i])

In [145]:
irl['Neighbourhood'] = v

In [146]:
#irl.loc[irl['Address'].str.contains("No.1 Windtown Park")]

In [147]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    if commas1[i] in b:
        str1 = commas1[i]
        dig = lastnum(str1)
        if str1[:4] == "Rear":
            str2 = str1[dig:]
            v.append(str2.strip())
        else:
            v.append(str1.strip())
    else:
        v.append(commas1[i])

In [148]:
irl['Neighbourhood'] = v

In [149]:
#irl.loc[irl['Address'].str.contains("Rear 16 Prince Edward Terrace")]

In [150]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    if commas1[i] in b:
        str1 = commas1[i]
        dig = lastnum(str1)
        if str1[:4] == "Rere":
            str2 = str1[dig:]
            v.append(str2.strip())
        else:
            v.append(str1.strip())
    else:
        v.append(commas1[i])

In [151]:
irl['Neighbourhood'] = v

In [152]:
irl.loc[irl['Neighbourhood'].str.contains("22 Leinster Road"), 'Neighbourhood'] = 'Leinster Road'

In [153]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    if commas1[i] in b:
        str1 = commas1[i]
        dig = lastnum(str1)
        if str1[:4] == "Unit":
            str2 = str1[dig:]
            v.append(str2.strip())
        else:
            v.append(str1.strip())
    else:
        v.append(commas1[i])

In [154]:
irl['Neighbourhood'] = v

In [155]:
#irl.loc[irl['Address'].str.contains("Unit 16 Lios Na Mara")]

In [156]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    if commas1[i] in b:
        str1 = commas1[i]
        dig = lastnum(str1)
        if str1[:5] == "Units":
            str2 = str1[dig:]
            v.append(str2.strip())
        else:
            v.append(str1.strip())
    else:
        v.append(commas1[i])

In [157]:
irl['Neighbourhood'] = v

In [158]:
#irl.loc[irl['Address'].str.contains("Units 1-6 Clonattin Lodge")]

In [159]:
commas1 = np.array(irl['Neighbourhood'])
no = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if has_numbers(str1):
        no.append(str1)
    else:
        continue

In [160]:
c = set(no)

In [161]:
len(c)

3715

In [162]:
#c

In [163]:
irl.loc[irl['Neighbourhood'].str.contains("-")]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
197,"Newhill, Two-Mile-Borris, Thurles",NaN,Tipperary,213000.0,1,1,1,2,2010,1,Thurles,South East (IE),0,Two-Mile-Borris,Munster,Other
8294,"21-22 Duffry Gate, Enniscorthy",NaN,Wexford,236000.0,1,1,1,1,2010,9,Enniscorthy,South East (IE),0,21-22 Duffry Gate,Leinster,Estate
15192,"1-3 (Inclusive) Congress Terrace, Longford Town",NaN,Longford,340000.0,1,1,1,2,2011,4,Longford Town,Midlands,0,1-3 (Inclusive) Congress Terrace,Leinster,Estate
15639,"10-11 Lower Tinahask, Arklow",NaN,Wicklow,105000.0,1,1,1,2,2011,5,Arklow,Mid-East,0,10-11 Lower Tinahask,Leinster,Estate
16079,"6 And 7 An T-Uachtarlann, Clones",NaN,Monaghan,160000.0,1,1,1,1,2011,5,Clones,Border,0,And 7 An T-Uachtarlann,Ulster,Estate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285593,"1-4 Davis Hall, Lucan, County Dublin",NaN,Dublin,1221720.0,0,0,0,1,2019,12,Lucan,Dublin,0,1-4 Davis Hall,Leinster,Estate
285594,"1-4 Ross Gate, Lucan, County Dublin",NaN,Dublin,1221720.0,0,0,0,1,2019,12,Lucan,Dublin,0,1-4 Ross Gate,Leinster,Estate
285595,"1-4 Wilkes Court, Lucan, County Dublin",NaN,Dublin,1221720.0,0,0,0,1,2019,12,Lucan,Dublin,0,1-4 Wilkes Court,Leinster,Estate
285677,"17 Coundon Court (Jabalal-Dukhan), Killiney, C...",NaN,Dublin,950000.0,1,1,1,2,2021,1,Killiney,Dublin,0,Coundon Court (Jabalal-Dukhan),Leinster,Estate


In [164]:
irl.loc[irl['Neighbourhood'].str.contains("Glebe 1"), 'Neighbourhood'] = 'Glebe'

In [165]:
irl.loc[irl['Neighbourhood'].str.contains("Glebe 2"), 'Neighbourhood'] = 'Glebe'

In [166]:
irl.loc[irl['Neighbourhood'].str.contains("Jabalal-Dukhan"), 'Neighbourhood'] = 'Coundon Court'
irl.loc[irl['Neighbourhood'].str.contains("2D") & irl['County'].str.contains('Kerry'), 'Neighbourhood'] = 'Ballyard'
irl.loc[irl['Neighbourhood'].str.contains("Ballyard") & irl['County'].str.contains('Kerry'), 'Neighbourhood'] = 'Ballyard'
irl.loc[irl['Neighbourhood'].str.contains("Ballyard") & irl['County'].str.contains('Kerry'), 'Neighbourhood'] = 'Ballyard'
irl.loc[irl['Neighbourhood'].str.contains('Kiltoom')  & irl['County'].str.contains('Roscommon'), 'Neighbourhood'] = 'Kiltoom'

In [167]:
irl.drop(irl[irl['Neighbourhood'].str.contains("& 7 James St Id 7 Is 3158607Dh")].index, inplace=True)

In [168]:
commas1 = np.array(irl['Neighbourhood'])
no = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if has_numbers(str1):
        no.append(str1)
    else:
        continue

In [169]:
c = set(no)

In [170]:
len(c)

3711

In [171]:
#v

In [172]:
irl.loc[irl['Address'].str.contains("123A"), 'Neighbourhood'] = 'Cedarfield'

In [173]:
irl.loc[irl['Address'].str.contains("13A Bowdens")]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
20712,"12A & 13A Bowdens Row, Castlecomer",NaN,Kilkenny,75000.0,1,1,1,1,2011,10,Castlecomer,South East (IE),0,& 13A Bowdens Row,Leinster,Estate


In [174]:
#c

In [175]:
commas1 = np.array(irl['Neighbourhood'])
no = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if has_numbers(str1):
        no.append(str1)
    else:
        continue

In [176]:
c = set(no)

In [177]:
len(c)

3710

In [178]:
irl.shape

(306501, 16)

In [179]:
irl.drop(irl[irl['Neighbourhood'].isin(c)].index, inplace=True)

In [180]:
irl.shape

(302695, 16)

In [181]:
irl['Neighbourhood'].nunique()

62192

In [182]:
commas1 = np.array(irl['Neighbourhood'])
new = []
for i in range(len(commas1)):
    str1 = commas1[i]
    res = " ".join(str1.split())
    new.append(res)

In [183]:
#new

In [184]:
irl['Neighbourhood'] = new

In [185]:
irl['Neighbourhood'].nunique()

59876

In [186]:
#irl[irl['Town'].str.contains('Naas')]

In [187]:
#c

In [188]:
irl.shape

(302695, 16)

In [189]:
#irl.loc[irl['County'].str.contains('Roscommon')].head(20)

In [190]:
irl.loc[irl['County'].str.contains('Roscommon') & irl['Neighbourhood'].str.contains('Warren'), 'Neighbourhood'] = 'Warren'

In [191]:
irl.loc[irl['Neighbourhood'].str.contains('Marian') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Marian Road'

In [192]:
irl.loc[irl['Neighbourhood'].str.contains('Grange') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Grange'

In [193]:
irl.loc[irl['Neighbourhood'].str.contains('Silver') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Silveroe'

In [194]:
irl.loc[irl['Neighbourhood'].str.contains('Mocmoyne') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Mocmoyne'

In [195]:
irl.loc[irl['Neighbourhood'].str.contains('Meadowvale') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Meadow Vale'

In [196]:
irl.loc[irl['Neighbourhood'].str.contains('Abbey') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Abbeytown' 

In [197]:
irl.loc[irl['Neighbourhood'].str.contains('Felton') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Felton'

In [198]:
irl.loc[irl['Neighbourhood'].str.contains('Ballymore') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Ballymore'

In [199]:
irl.loc[irl['Neighbourhood'].str.contains('Marion') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Marian Road'

In [200]:
irl.loc[irl['Neighbourhood'].str.contains('Maple') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Maple Drive'

In [201]:
irl.loc[irl['Neighbourhood'].str.contains('') & irl['Town'].str.contains('Boyle')]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
260,"Srabra, Lough Allen, Boyle",NaN,Roscommon,5500.00,0,1,1,2,2010,1,Boyle,West,0,Lough Allen,Connacht,Other
969,"No 2 Lindenwood, Cootehall, Boyle",NaN,Roscommon,203832.59,1,0,0,2,2010,2,Boyle,West,0,Cootehall,Connacht,Other
1327,"4 Warren Way, Boyle",NaN,Roscommon,160000.00,1,1,1,1,2010,2,Boyle,West,0,Warren,Connacht,Estate
1611,"12 Esker Grove, Cootehall, Boyle",NaN,Roscommon,187000.00,1,0,0,2,2010,2,Boyle,West,0,Cootehall,Connacht,Estate
1940,"No. 20 Forest Park Manor, Carrick Road, Boyle",NaN,Roscommon,182697.15,1,0,0,1,2010,3,Boyle,West,0,Carrick Road,Connacht,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302321,"Spa, Boyle, Roscommon",NaN,Roscommon,55000.00,1,1,1,1,2016,2,Boyle,West,0,Spa,Connacht,Other
302348,"Ballyfarnon, Boyle, Roscommon",NaN,Roscommon,61000.00,1,1,1,1,2017,7,Boyle,West,0,Ballyfarnon,Connacht,Other
302522,"Cashel, Boyle, Co Roscommon",NaN,Roscommon,80000.00,1,1,1,1,2017,3,Boyle,West,0,Cashel,Connacht,Other
302525,"Spa, Boyle, Co Roscommon",NaN,Roscommon,47000.00,1,1,1,2,2019,4,Boyle,West,0,Spa,Connacht,Other


In [202]:
boy = irl.loc[irl['Town'].str.contains("Boyle")]
df10 = pd.DataFrame(boy['Neighbourhood'].unique())
#df10.to_csv('boyle_unique.csv')

In [203]:
irl.loc[irl['Neighbourhood'].str.contains('Moc') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Mocmoyne'

In [204]:
irl.loc[irl['Neighbourhood'].str.contains('Covvigeenroe') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Corrigeenroe'

In [205]:
irl.loc[irl['Neighbourhood'].str.contains('Abbay') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Abbeytown' 

In [206]:
irl.loc[irl['Neighbourhood'].str.contains('Coot') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Cootehall'

In [207]:
commas1 = np.array(irl['Neighbourhood'])
v = []
for i in range(len(commas1)):
    str1 = commas1[i]
    #dig = lastnum(str1)
    if str1[:8] == "Property":
        v.append(str1)
    else:
        continue

In [208]:
v

['Property At Currabinny',
 'Property At Balrath Road',
 'Property At Munigelly',
 "Property At Buck Leary's Cross",
 'Property At Main Street',
 'Property At Harristown',
 'Property At Killeenyarda',
 'Property @ Tivanagh',
 'Property @ Kilkeasey',
 'Property']

In [209]:
commas1 = np.array(irl['Neighbourhood'])
z = []
for i in range(len(commas1)):
    str1 = commas1[i]
    #dig = lastnum(str1)
    if str1[:5] == "Lands":
        z.append(str1)
    else:
        continue

In [210]:
#z

In [211]:
l = v + z

In [212]:
irl.drop(irl[irl['Neighbourhood'].isin(l)].index, inplace=True)

In [213]:
irl.shape

(302635, 16)

In [214]:
boy = irl.loc[irl['County'].str.contains("Roscommon")]
df10 = pd.DataFrame(boy['Neighbourhood'].unique())
#df10.to_csv('roscommon_unique.csv')

In [215]:
irl.loc[irl['Neighbourhood'].str.contains('Knockvi') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Knockvicar'

In [216]:
irl.loc[irl['Neighbourhood'].str.contains('Elpin') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Elphin Street'

In [217]:
irl.loc[irl['Neighbourhood'].str.contains('Ballinaneen') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Ballinameen' 

In [218]:
irl.loc[irl['Neighbourhood'].str.contains('Cloonloogh') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Cloonloo' 

In [219]:
irl.loc[irl['Neighbourhood'].str.contains('St. Josephs') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'St. Josephs' 

In [220]:
irl.loc[irl['Neighbourhood'].str.contains('Keadue') & irl['Town'].str.contains('Boyle'), 'Neighbourhood'] = 'Keadue'

In [221]:
irl.loc[irl['Neighbourhood'].str.contains('Waterford City'), 'Neighbourhood'] = 'Doyle Street'

In [222]:
val = pd.Series(irl['Neighbourhood'].value_counts())

In [223]:
val[:50]

Main Street                2285
Dublin Road                1902
Station Road                829
Church Road                 618
Johnstown                   594
Church Street               556
Millers Glen                492
Castletroy                  462
Shackleton Park             417
Dooradoyle                  400
Abbeyside                   371
Churchfields                365
Ridgewood                   349
Newtown                     342
Raheen                      332
Blackrock                   313
Clay Farm                   311
Ennis Road                  310
Barrack Street              305
High Street                 300
Grange                      299
Golf Links Road             298
Killarney Road              294
Marina Village              292
Bridge Street               280
Tullow Road                 275
Robswall                    274
Beacon South Quarter        271
Adamstown Square            269
Woodlands                   266
Adamstown                   256
Castleto

In [224]:
irl.loc[irl['Neighbourhood'] == 'Sligo']

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
289065,"4 Moyvale Park, Sligo, Ballina",NaN,Mayo,207000.0,1,1,1,1,2011,12,Ballina Mayo,West,0,Sligo,Connacht,Estate


In [225]:
l = list(irl['Town'].unique())
fix_neigh = np.array(irl['Neighbourhood'])
change = []
for i in range(len(fix_neigh)):
    if fix_neigh[i] in l and fix_neigh[i] not in change:
        change.append(fix_neigh[i])
    else:
        continue

In [226]:
irl.loc[irl['Neighbourhood'].isin(change)].head()

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
3872,"Number 33, Choill Darach Bree, Castleblayney",NaN,Monaghan,168894.27,0,0,0,2,2010,5,Castleblayney,Border,0,Bree,Ulster,Other
6072,"Apt. 5 Shanagarry, Milltown",Dublin 6,Dublin,170000.00,1,1,1,1,2010,7,Milltown Dublin,Dublin,1,Shanagarry,Leinster,Other
10505,"Unit 1 2 & 7 Derrinturn, Carbury",NaN,Kildare,115000.00,1,1,1,1,2010,10,Carbury Kildare,Mid-East,0,Derrinturn,Leinster,Other
20066,"The Nevitt Knockvicar, Boyle",NaN,Roscommon,80000.00,1,1,1,2,2011,10,Boyle,West,0,Knockvicar,Connacht,Other
24494,"64. Bree, Castleblayney",NaN,Monaghan,90000.00,0,1,1,1,2012,2,Castleblayney,Border,0,Bree,Ulster,Estate


In [227]:
#change

In [228]:
irl.shape

(302635, 16)

In [229]:
#double

In [230]:
double_name[double_name['Neighbourhood'].str.contains('Longford')]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood
76448,"Longford, Castlerea",NaN,Roscommon,35000.0,0,1,1,2,2015,5,Castlerea,West,0,Longford
209452,"Longford, Duleek, Meath",NaN,Meath,225000.0,1,1,1,1,2015,10,Duleek,Mid-East,0,Longford
251929,"10 Oaklands Grove, Longford, Kildare",NaN,Kildare,149000.0,0,1,1,1,2020,10,Kildare,Mid-East,0,Longford


In [231]:
commas1 = np.array(double_name['Neighbourhood'])
v = []
for i in range(len(commas1)):
    str1 = commas1[i]
    if str1[0].isnumeric():
        dig = lastnum(str1)
        v.append(str1[dig:].strip())
    else:
        v.append(str1.strip())

In [232]:
irl['Neighbourhood'].nunique()

59808

### Add in corresponding town to area
There will be multiple neighbourhoods designated as "Main Street" for example so I will add the corresponding towns to each neighbourhood. 

In [233]:
commas1 = np.array(irl['Neighbourhood'])
town = np.array(irl['Town'])
no = []
for i in range(len(town)):
    new = commas1[i] + " " + town[i]
    no.append(new)

In [234]:
irl['Neighbourhood'] = no

In [235]:
val1 = pd.Series(irl['Neighbourhood'].value_counts())
val1[:50]

Johnstown Navan                          501
Millers Glen Swords                      490
Castletroy Limerick                      461
Shackleton Park Lucan                    417
Dooradoyle Limerick                      397
Abbeyside Dungarvan Waterford            365
Ridgewood Swords                         347
Clay Farm Leopardstown                   310
Churchfields Ashbourne                   295
Robswall Malahide                        274
Beacon South Quarter Sandyford           270
Tullow Road Carlow                       269
Adamstown Square Adamstown Dublin        252
Old Quarter Ballincollig                 251
Dublin Road Maynooth                     241
Belmayne Balgriffin                      240
Rockbrook Sandyford                      236
Adamstown Lucan                          231
Marina Village Greystones                231
Newcastle Road Lucan                     231
Phoenix Park Racecourse Castleknock      230
Kilminchy Portlaoise                     229
Barnageera

In [236]:
irl.loc[irl['Neighbourhood'].str.contains('Abbeyside Dungarvan Waterford City')]

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate


In [237]:
#irl.to_csv('neigh-again-1.csv')

## Take away values with Neighbourhood Value Counts equal to 1 - may get rid of typos

In [238]:
counts = irl['Neighbourhood'].value_counts()

res11 = irl[~irl['Neighbourhood'].isin(counts[counts <=  1].index)]

In [239]:
res11

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
0,"5 Braemor Drive, Churchtown",NaN,Dublin,343000.0,1,1,1,0,2010,1,Churchtown Dublin,Dublin,0,Braemor Drive Churchtown Dublin,Leinster,Estate
1,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,185000.0,1,0,0,1,2010,1,Portlaoise,Midlands,0,Summerhill Lane Portlaoise,Leinster,Estate
3,"1 The Haven, Mornington",NaN,Meath,400000.0,1,1,1,1,2010,1,Mornington,Mid-East,0,The Haven Mornington,Leinster,Estate
4,"12 Sallymount Avenue, Ranelagh",NaN,Dublin,425000.0,1,1,1,1,2010,1,Ranelagh,Dublin,0,Sallymount Avenue Ranelagh,Leinster,Estate
5,"15 Carriglawn, Waterpark, Carrigaline",NaN,Cork,180000.0,1,1,1,1,2010,1,Carrigaline,South-West (IE),0,Waterpark Carrigaline,Munster,Estate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306895,"Ballinkillin, Bagenalstown, Carlow",NaN,Carlow,175000.0,1,1,1,1,2017,6,Muine Bheag,South East (IE),0,Ballinkillin Muine Bheag,Leinster,Other
306896,"Donore, Bagenalstown, Co. Carlow",NaN,Carlow,148000.0,1,1,1,1,2013,4,Muine Bheag,South East (IE),0,Donore Muine Bheag,Leinster,Other
306898,"Loughlinstown, Rathoath, Co Meath",NaN,Meath,610000.0,1,0,0,2,2013,5,Ratoath,Mid-East,0,Loughlinstown Ratoath,Leinster,Other
306899,"5 Streamstown, Rathoath, Co Meath",NaN,Meath,380000.0,1,1,1,1,2017,12,Ratoath,Mid-East,0,Streamstown Ratoath,Leinster,Estate


In [240]:
res11.dtypes

Address                       object
Postal Code                   object
County                        object
Price (€)                    float64
Not Full Market Price          int64
VAT Exclusive                  int64
Description of Property        int64
Property Size Description      int64
Year                           int64
Month                          int64
Town                          object
Region                        object
Apartment                      int64
Neighbourhood                 object
Province                      object
Estate                        object
dtype: object

In [241]:
res11[res11['Address'] == '53 Mounttown Rd Lower, Dun Laoghaire, Dublin']

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate
286170,"53 Mounttown Rd Lower, Dun Laoghaire, Dublin",NaN,Dublin,525000.0,1,1,1,1,2019,2,Dún Laoghaire,Dublin,0,Mounttown Rd Lower Dún Laoghaire,Leinster,Estate


In [242]:
xz = res11.loc[res11['Neighbourhood'].str.contains("Rd")]

In [243]:
#print(s.replace(' ', '-'))
st = np.array(xz['Neighbourhood'])
vals = []
for i in range(len(st)):
    string = st[i].replace("Rd", 'Road')
    vals.append(string)

In [244]:
#vals
xz['Neighbourhood'] = vals


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [245]:
adds = list(xz['Address'])

In [246]:
res11.shape

(255735, 16)

In [247]:
res11.drop(res11[res11['Address'].isin(adds)].index, inplace=True)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [248]:
res11.shape

(255418, 16)

In [249]:
res12 = pd.concat([res11, xz], ignore_index=True)

In [250]:
res12.shape

(255735, 16)

In [251]:
res12.to_csv('more-than1-neigh.csv')